# Example for utilization of powerfactory-tools -- Export

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import logging
import pathlib

from powerfactory_tools.exporter import PowerFactoryExporter
from powerfactory_tools.exporter import export_powerfactory_data
from psdm.steadystate_case.case import Case as SteadystateCase
from psdm.topology.topology import Topology
from psdm.topology_case.case import Case as TopologyCase

In [6]:
PROJECT_NAME = r"7 Tests\PowerFactory-Tools"  # may be also full path "dir_name\project_name"
EXPORT_PATH = pathlib.Path("export")
PF_USER_PROFILE = "krahmers"  # specification may be necessary
PYTHON_VERSION = "3.10"
STUDY_CASES = ["Base", "Outage", "Industry_Park"]

## Export using an exporter instance

In [ ]:
# The exporter generates for each study case three JSON files: topology, topology_case, steadystate_case.
# Here, only active grids will be exported.

with PowerFactoryExporter(
    project_name=PROJECT_NAME,
    powerfactory_user_profile=PF_USER_PROFILE,
    python_version=PYTHON_VERSION,
    logging_level=logging.INFO,
) as exporter:
    # Export first two study cases.
    exporter.export(
        export_path=EXPORT_PATH,
        study_case_names=STUDY_CASES[0:2],
    )

    # Switch to the study case "Industry_Park", which consits of 2 active grids and save the exported data in separate folder
    exporter.export(
        export_path=EXPORT_PATH / pathlib.Path("Industry_Park"),
        study_case_names=STUDY_CASES[2:],
    )

    # Export just the study case that is active at the moment
    # exporter.export(export_path=EXPORT_PATH)

## Export using the export function running in a new process with default parameters

In [ ]:
# As the export function is executed in a process that is terminated after execution, the PowerFactory API is also closed.

export_powerfactory_data(
    export_path=EXPORT_PATH,
    project_name=PROJECT_NAME,
    study_case_names=STUDY_CASES,
    powerfactory_user_profile=PF_USER_PROFILE,
    python_version=PYTHON_VERSION,
)

### [Optional] Display example JSON files

In [ ]:
t = Topology.from_file(EXPORT_PATH / f"{GRID_NAME}_topology.json")
t_s = t.model_dump()
print(json.dumps(t_s, sort_keys=True, default=str, indent=2))

In [ ]:
tc = TopologyCase.from_file(EXPORT_PATH / f"{GRID_NAME}_topology_case.json")
tc_s = tc.model_dump()
print(json.dumps(tc_s, sort_keys=True, default=str, indent=2))

In [ ]:
ssc = SteadystateCase.from_file(EXPORT_PATH / f"{GRID_NAME}_steadystate_case.json")
ssc_s = ssc.model_dump()
print(json.dumps(ssc_s, sort_keys=True, default=str, indent=2))

### [Optional] Display raw JSON schema of Power System Data Model

In [ ]:
t_s = Topology.model_json_schema()
print(json.dumps(t_s, sort_keys=True, default=str, indent=2))

In [ ]:
tc_s = TopologyCase.model_json_schema()
print(json.dumps(tc_s, sort_keys=True, default=str, indent=2))

In [ ]:
ssc_s = SteadystateCase.model_json_schema()
print(json.dumps(ssc_s, sort_keys=True, default=str, indent=2))